# EAZY SED Composite Evolution Code

- This code is intended to pull galaxies selected from analysis to be used in the EAZY SED Composite Evolution Code.
    -  we will pull the restframe SEDs of galaxies and do some of the following things.



1. We will first have two seperate seds we choose to look for then read in:
    - The ugr seds (the averages from each redshift range)
    - the UVJ seds (the average sed from each galaxy region)

2. We can then create the composite sets and output the plots of these galaxies

In [1]:
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
import pandas as pd
import os
from astLib import astSED
import astropy.io.fits as fits
from carf import * # custom module for functions relating to the project
import matplotlib.path as mpath
import seaborn as sns
from scipy.stats import gaussian_kde

# So that we can change the helper functions without reloading the kernel
%load_ext autoreload
%autoreload 2

In [4]:
# Readin the two dataframes
uvj_seds = pd.read_csv('outputs\ThesisPlots\ZFOURGE_obsevational_composites_fluxesType1AGN_EAZY_UVJ_galaxies.csv')

# Drop the unnamed column
uvj_seds = uvj_seds.drop(columns='Unnamed: 0')

In [5]:
uvj_seds

,id,z,population
0,COSMOS_1341,1.2650,SF
1,CDFS_18026,0.7203,Quiescent
2,CDFS_18214,0.7346,Dusty


In [13]:
# Read in the ugr data
ugr_seds = pd.read_csv('outputs\ThesisPlots\ZFOURGE_obsevational_composites_fluxesType1AGN_EAZY_ugr_galaxies.csv')

# Drop the unnamed column
ugr_seds = ugr_seds.drop(columns='Unnamed: 0')

In [11]:
ugr_seds

,ID,Redshift,Bin
0,COSMOS_1698,1.2438,z < 2.6
1,UDS_1597,2.9778,2.6 < z < 3.5
2,UDS_15263,3.7467,z > 3.5
